In [1]:
import gudhi
import numpy as np

from functions.data_loading import get_filtered_data_range, get_data_range, get_flight_persistence
from functions.data_filtering import complete_flight_filter
from functions.data_filtering import filter_by_bools, filter_flights

from datetime import datetime
from collections import Counter
from tqdm import tqdm
from traffic.core import Traffic, Flight

In [4]:
origin = "bergen"
destination = "oslo"

unfiltered_data, file_name = get_data_range(origin, destination, datetime(year=2023, month=1, day=1), datetime(year=2024, month=1, day=1))
filtered_flights, file_name = get_filtered_data_range(unfiltered_data, file_name, complete_flight_filter(origin, destination))
persistences, _ = get_flight_persistence(filtered_flights, file_name)

data/complete_flights/oslo-bergen-2023-01-01-2024-01-01.pkl


TypeError: boolean value of NA is ambiguous

In [ ]:
circly_flights=[]
for i, tree in enumerate(persistences):
    persistence = tree.persistence()

    more_than_00001 = len([x for x in persistence if  x[0] == 1 and x[1][1] - x[1][0] > 0.0001])
    #more_than_00005 = len([x for x in persistence if  x[0] == 1 and x[1][1] - x[1][0] > 0.0005])
    if more_than_00001 > 0:
        circly_flights.append(i)

In [ ]:
"""
codes = Counter()
for flight in circly_flights:
    codes.update(filtered_flights[flight].squawk)

codes
"""

In [ ]:
bool_array = np.zeros(len(persistences))
for i in circly_flights:
    bool_array[i] = 1

loopy_flights = filter_flights(filter_by_bools(bool_array), filtered_flights)
loopy_flights.map_leaflet()

In [ ]:
loopy_flights[38].map_leaflet()

In [ ]:
flight_amount = len(persistences)
condensed_distance_matrix = []

for i in tqdm(range(flight_amount)):
    for j in range(i+1, flight_amount):
        tree_i = persistences[i]
        tree_j = persistences[j]
        #pers_i = [x for x in tree_i.persistence_intervals_in_dimension(1) if  x[1] - x[0] > 0.0001]
        #pers_j = [x for x in tree_j.persistence_intervals_in_dimension(1) if  x[1] - x[0] > 0.0001]
        pers_i = tree_i.persistence_intervals_in_dimension(1)
        pers_j = tree_j.persistence_intervals_in_dimension(1)
        dist = gudhi.bottleneck_distance(pers_i, pers_j, 0.0001)
        condensed_distance_matrix.append(dist)

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster

Z = linkage(condensed_distance_matrix, "complete")
clustering = fcluster(Z, t=0.002, criterion='distance', depth=2)

In [ ]:
for i in range(1, len(np.unique(clustering))+1):
    print(f"{i}: {sum(clustering == i)}")

In [ ]:
bool_array = clustering == 3

loopy_flights = filter_flights(filter_by_bools(bool_array), filtered_flights)
print(len(loopy_flights))
loopy_flights.map_leaflet()